In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import numpy as np
import os


In [ ]:
# Load the data
data_dir = 'path_to_your_data'  # replace with the path to your data
text = ''
for filename in os.listdir(data_dir):
    with open(os.path.join(data_dir, filename), 'r') as f:
        text += f.read()

In [ ]:
# Preprocess the data
chars = sorted(list(set(text)))
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# Prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, len(text) - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_index[char] for char in seq_in])
    dataY.append(char_to_index[seq_out])
n_patterns = len(dataX)

In [ ]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(len(chars))

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# Define the model
model = Sequential([
    GRU(256, input_shape=(X.shape[1], X.shape[2])),
    Dense(y.shape[1], activation='softmax')
])

In [ ]:

# Compile and fit the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128)
